In [ ]:
from graph_gaussian_process import util
import matplotlib as mpl
from matplotlib import pyplot as plt
import networkx as nx
import numpy as np

mpl.rcParams["figure.dpi"] = 144

Considering Gaussian processes on directed acyclic graphs is a general approach to reduce the computational burden for inference. However, often, we are interested in graph structures that capture the spatial relationships between nodes. The function `lattice_neighborhoods` does just that. It constructs a directed graph for nodes on a lattice, ensuring that the graph is acyclic. Two options are supported: Cuboidal and ellipsoidal receptive fields. The latter are preferable because they preserve the isotropy of the space we are trying to approximate. They are also more efficient because the volume of a hyperellipsoid is strictly smaller than a hypercube with the same diameter, reducing the number of nodes in the receptive field. For example, in three dimensions, the relative volume of a sphere and cube is
$$
\frac{V_\mathrm{sphere}}{V_\mathrm{cube}}=\frac{4\pi r^3 / 3}{\left(2 r\right)^3}\approx 0.52.
$$
The number of nodes is reduced by a factor of two. But, because matrix inversion required to evaluate the likelihood scales approximately cubicly with the number of nodes, we can reduce the computational cost by almost an order of magnitude. 

The cell below illustrates the receptive fields in two dimensions and shows both predecessors (that the example node depends on in the likelihood) and successors (that depend on the example node). The union of predecessors and successors is the receptive field of the node.

In [ ]:
# Define the lattice size, receptive field size, and choose example node positions.
width, height = 10, 15
k = (3, 2)
x, y = 5, 7

shape = (width, height)
node = np.ravel_multi_index((x, y), shape)
coords = util.coordgrid(np.arange(width), np.arange(height))

# For each of the two receptive field methods, ...
fig, axes = plt.subplots(1, 2, sharex=True, sharey=True)
for ax, bounds in zip(axes, ["cuboid", "ellipsoid"]):
    # Plot the lattice and example node.
    ax.scatter(*coords.T, color="silver", marker=".")
    ax.scatter(*coords[node], label="example node", zorder=99).set_edgecolor("w")
    
    # Get the lattice neighborhoods and construct a directed graph.
    neighborhoods = util.lattice_neighborhoods((width, height), k, bounds=bounds)
    edge_index = util.neighborhood_to_edge_index(neighborhoods, indexing="numpy")
    graph = util.edge_index_to_graph(edge_index)
    
    # Show predecessors (that the example node depends on in the likelihood) and successors 
    # (that depend on the example node in the likelihood).
    for label in ["predecessors", "successors"]:
        # Get the nodes and ensure they satisfy the ordering constraint.
        nodes = np.asarray(list(getattr(graph, label)(node)))
        if label == "predecessors":
            assert (nodes <= node).all()
        else:
            assert (nodes >= node).all()
        
        # Plot the nodes.
        zorder = 10 if label == "predecessors" else 8
        ax.scatter(*coords[nodes].T, label=label, zorder=zorder).set_edgecolor("w")
        edges = graph.out_edges(node) if label == "successors" else graph.in_edges(node)
        edges = nx.draw_networkx_edges(graph, coords, edges, ax=ax, node_size=0, alpha=0.5, 
                                       arrowsize=7)
        for edge in edges:
            edge.set_zorder(9)
            
    ax.set_aspect("equal")
    ax.set_title(f"{bounds} bounds")
    ax.set_xlabel("Position $x_1$")
        
axes[0].set_ylabel("Position $x_2$")
axes[0].yaxis.set_major_locator(mpl.ticker.MaxNLocator(integer=True))
axes[0].legend(loc="upper right")
fig.tight_layout()